<a href="https://colab.research.google.com/github/SnehaVPujari007/Crypto-Data-Analysis/blob/main/Crypto_Currency_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd

In [18]:
import requests

# API endpoint
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 50,
    "page": 1,
    "sparkline": False
}



In [ ]:
# Fetch data
response = requests.get(url, params=params)
data = response.json()

# Extract required fields
crypto_data = [
    {
        "name": coin["name"],
        "symbol": coin["symbol"],
        "current_price": coin["current_price"],
        "market_cap": coin["market_cap"],
        "volume_24h": coin["total_volume"],
        "price_change_24h": coin["price_change_percentage_24h"]
    }
    for coin in data
]


In [19]:
top_5 = sorted(crypto_data, key=lambda x: x["market_cap"], reverse=True)[:5]
top_5 = pd.DataFrame(top_5)
print(top_5)

       name symbol  current_price     market_cap   volume_24h  \
0   Bitcoin    btc      96936.000  1916689917818  92444442828   
1  Ethereum    eth       3134.790   377670267659  34216696002   
2    Tether   usdt          1.001   130400176332  60069321161   
3    Solana    sol        240.630   114365678232   9307416053   
4       BNB    bnb        609.960    88940993569   1729973640   

   price_change_24h  
0           5.10327  
1           1.12770  
2           0.00400  
3           3.16777  
4          -0.47018  


In [20]:
avg_price = sum([coin["current_price"] for coin in crypto_data]) / len(crypto_data)
print(avg_price)

4240.7583859356


In [22]:
highest_change = max(crypto_data, key=lambda x: x["price_change_24h"])
lowest_change = min(crypto_data, key=lambda x: x["price_change_24h"])

print(f"Highest price change: {highest_change} ")
print(f"Highest price change: {lowest_change} ")

Highest price change: {'name': 'Bitcoin Cash', 'symbol': 'bch', 'current_price': 485.6, 'market_cap': 9563030042, 'volume_24h': 750510296, 'price_change_24h': 11.27945} 
Highest price change: {'name': 'Bonk', 'symbol': 'bonk', 'current_price': 5.114e-05, 'market_cap': 3848212271, 'volume_24h': 2383745042, 'price_change_24h': -8.79811} 


In [23]:
pip install gspread oauth2client


In [25]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
import time




In [26]:
# Authenticate with Google Sheets

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive",
         'https://www.googleapis.com/auth/spreadsheets']
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/melodic-eye-425616-e1-7cce7250f0a4.json", scope)
client = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1oSlCyyUgCVWJ14caOcYVMfL2ZFLot0x3yoJveeVVvRI/edit?usp=sharing"

spreadsheet_key = "1oSlCyyUgCVWJ14caOcYVMfL2ZFLot0x3yoJveeVVvRI"
sheet = client.open_by_key(spreadsheet_key).sheet1



In [27]:
#Fetch Cryptocurrency Data
def fetch_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 50,
        "page": 1,
        "sparkline": False,
    }
    response = requests.get(url, params=params)
    return response.json()



In [28]:
# Update Data in Google Sheets
def update_sheet():
    # Fetch data
    data = fetch_crypto_data()

    # Clear existing sheet data
    sheet.clear()

    # Add headers
    headers = ["Name", "Symbol", "Price (USD)", "Market Cap", "Volume (24h)", "Price Change (24h %)"]
    sheet.append_row(headers)

    # Add cryptocurrency data
    for coin in data:
        row = [
            coin["name"],
            coin["symbol"],
            coin["current_price"],
            coin["market_cap"],
            coin["total_volume"],
            coin["price_change_percentage_24h"],
        ]
        sheet.append_row(row)



In [ ]:
# Run Updates Periodically
while True:
    update_sheet()
    print("Google Sheet updated.")
    time.sleep(300)  # Update every 5 minutes

Google Sheet updated.
Google Sheet updated.
